In [4]:
from datetime import datetime

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer

from nltk.tokenize import sent_tokenize

import langchain
from langchain.cache import InMemoryCache


In [5]:
def filter_summary(text,text_array,MAX_WORD_COUNT):

    word_count=0
    summary=' '.join(text_array)

    summary_l = sent_tokenize(summary)
    text_l = complete_quote_sentence(text)

    summary_l = arrange_sentence_sequence(text,text_l,summary_l)

    output_l = output_quote_filler(text,summary_l)

    add_ellip = add_ellipsis(text_l,output_l)

    summary_limited= str()
    ct=0
    ELLIP=False
    for i in add_ellip:
      words=len(i.replace('...','').split())
      if word_count+words < MAX_WORD_COUNT:

          if not '...' in i and ELLIP: #add ellips while skipping sentence long sentence
              summary_limited+=' ... '

          summary_limited+=i+' '
          word_count+=words
      else: #skipp long sentence
          ELLIP=True

    if summary_limited=='':
        summary_limited=add_ellip[0]

    return summary_limited

def complete_quote_sentence(text):

    sl=sent_tokenize(text)
    # print_list(sl)

    ts=str()
    nl=list()

    for i in range(len(sl)):
        if '"' in sl[i]:
            ct = sl[i].count('"')
            # print(' " present in {} line for {} times'.format(i,ct))

            if ct%2 != 0:
                if len(ts)==0:
                    ts+=sl[i]
                else :
                    ts+=sl[i]
                    nl.append(ts)
                    ts=str()

            else:
                nl.append(sl[i])


        else:
            nl.append(sl[i])

    return nl

def arrange_sentence_sequence(text,text_l,summary_l):

    input_l = output_quote_filler(text,text_l)
    output_l = output_quote_filler(text,summary_l)

    sequence = list()

    for i in input_l:
        if i in output_l:
            sequence.append(i)

    return sequence

def print_list(l,new_line=1):
    print('\n'*new_line)
    c=0
    for i in l:
        print(c, ' : ',i)
        c+=1

def print_dict(d,new_line=1):
    print('\n'*new_line)
    c=0
    for k,i in d.items():
        print('{} : {}'.format(i,k))

def output_quote_filler(text,sum_l):

    sl=sent_tokenize(text)
    # sl=text_l
    # print_list(sl)

    ts=str()
    nl=list()
    nd=dict()

    num=0

    # print('\n\n')

    for i in range(len(sl)):
        if '"' in sl[i]:
            ct = sl[i].count('"')
            # print(' " present in {} line for {} times'.format(i,ct))

            if ct%2 != 0:
                if len(ts)==0:
                    ts+=sl[i]
                    # ti = i
                    nd[sl[i]]=num
                else :
                    ts+=sl[i]
                    nl.append(ts)
                    ts=str()
                    nd[sl[i]]=num
                    num+=1
                    # ti=0

            else:
                nl.append(sl[i])
                nd[sl[i]]=num
                num+=1


        else:
            nl.append(sl[i])
            nd[sl[i]]=num
            num+=1

    f_op = list()

    for i in sum_l:
        if i in nd.keys():
            f_op.append( nl[nd[i]] )

    return f_op

def add_ellipsis(input_text,output_text):

    s1_l= input_text
    s2_l= output_text

    if "..." in s1_l[0]:
       s= s1_l[0].replace(' ... ','')
       if s in s2_l:
          x=s2_l.index(s)
          s2_l[x]=s
        #   print('\n\n\n x : {} \n\n\n'.format(x))
    i=1
    s1_dic=dict()
    for s in s1_l:
        s1_dic[s]=i
        i+=1

    s2_dic=dict()
    for s in s2_l:
        s2_dic[s]=s1_dic[s.strip()]

    output_l=list()
    pre=s2_dic[s2_l[0]]
    i=-1

    for s in s2_l:
        i+=1
        # print(s)

        if i==0 and s2_dic[s]!=1:
            output_l.append('... '+s)
            pre =  s2_dic[s]

        elif pre == s2_dic[s]:
            output_l.append(''+s)
            pre =  s2_dic[s]

        else:

            if pre+1 == s2_dic[s]:
                output_l.append(''+s)
                pre =  s2_dic[s]
            else:
                output_l.append('... '+s)
                pre =  s2_dic[s]

    return output_l


In [6]:
text = '''Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hindi poet Harivansh Rai Bachchan and his wife, the social activist Teji Bachchan. He was educated at Sherwood College, Nainital, and Kirori Mal College, University of Delhi. His film career started in 1969 as a voice narrator in Mrinal Sen's film Bhuvan Shome. He first gained popularity in the early 1970s for films, such as Anand, Zanjeer, Roti Kapada Aur Makaan, and achieved greater stardom in later years, dubbed India's "angry young man" for several of his on-screen roles in Hindi films.[11][12] He consistently starred in top grossing Indian films with critical acclaim since the mid 1970s to 1980s, such as Deewaar, Sholay, Kabhi Kabhie, Hera Pheri, Amar Akbar Anthony, Parvarish, Kasme Vaade, Trishul, Don, Muqaddar Ka Sikandar, Suhaag, Dostana, Naseeb, Laawaris, Namak Halaal, Coolie, Sharaabi and Mard,[13][14] as well as some of his most acclaimed performances, include Namak Haraam, Abhimaan, Majboor, Mili, Chupke Chupke, Kaala Patthar, Shaan, Yaarana, Kaalia, Shakti, Aakhree Raasta, Shahenshah and Agneepath.[15][16] After taking a break from acting in the 1990s, his resurgence was marked in 2000 with Mohabbatein.[17] Since then he starred in several successful and acclaimed films like Kabhi Khushi Kabhie Gham, Aankhen, Baghban, Khakee, Black, Bunty Aur Babli, Sarkar, Kabhi Alvida Naa Kehna, Cheeni Kum, Shootout At Lokhandwala, Paa, Piku, Pink, Badla, Brahmāstra: Part One – Shiva and Kalki 2898 AD.[18] For Piku, he won his fourth National Film Award for Best Actor, making him the only actor to do so. Bachchan also made an appearance in a Hollywood film, The Great Gatsby (2013), in which he played a non-Indian Jewish characte'''


In [7]:
def LuhnSumy(text,number_of_sentences,MAX_WORD_COUNT):
  """_summary_

    Args:
          text (string): input text which you need to be summarized
          number_of_sentences (integer): number of sentence you want to summarize
    """

  model_name = 'LuhnSumy'
  # For Strings
  parser = PlaintextParser.from_string(text,Tokenizer("english"))
  # Using KL
  summarizer = LuhnSummarizer()
  #Summarize the document with 4 sentences
  summary = summarizer(parser.document,number_of_sentences)

  summary_text=[]
  for sentence in summary:
    summary_text.append(str(sentence))

  # summary_text = " ".join(summary_text)
  # return str(summary_text).strip()

  # return filter_summary(text,summary_text,MAX_WORD_COUNT),model_name
  return summary_text


In [13]:
summary = LuhnSumy(text, 3, 200)


In [14]:
summary


['Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hindi poet Harivansh Rai Bachchan and his wife, the social activist Teji Bachchan.',
 '[11][12] He consistently starred in top grossing Indian films with critical acclaim since the mid 1970s to 1980s, such as Deewaar, Sholay, Kabhi Kabhie, Hera Pheri, Amar Akbar Anthony, Parvarish, Kasme Vaade, Trishul, Don, Muqaddar Ka Sikandar, Suhaag, Dostana, Naseeb, Laawaris, Namak Halaal, Coolie, Sharaabi and Mard,[13][14] as well as some of his most acclaimed performances, include Namak Haraam, Abhimaan, Majboor, Mili, Chupke Chupke, Kaala Patthar, Shaan, Yaarana, Kaalia, Shakti, Aakhree Raasta, Shahenshah and Agneepath.',
 '[17] Since then he starred in several successful and acclaimed films like Kabhi Khushi Kabhie Gham, Aankhen, Baghban, Khakee, Black, Bunty Aur Babli, Sarkar, Kabhi Alvida Naa Kehna, Cheeni Kum, Shootout At Lokhandwala, Paa, Piku, Pink, Badla, Brahmāstra: Part One – Shiva and Kalki 2898 AD.']